<a href="https://colab.research.google.com/github/azzindani/03_Prompt_Engineering/blob/main/PE_EN_Qwen2_5_7B_4bit_Med_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Modules

In [1]:
!pip install -U -q bitsandbytes
!pip install -q optimum
!pip install -q auto-gptq

In [2]:
import torch

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  GenerationConfig,
  BitsAndBytesConfig,
  GPTQConfig
)

if torch.cuda.is_available():
  print('GPU is available!')
else:
  print('GPU is not available.')

GPU is available!


## Import Model

In [3]:
model_name = 'Qwen/Qwen2.5-7B'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_quant_type = 'nf4',
  bnb_4bit_compute_dtype = torch.float16,
  bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config = bnb_config,
  trust_remote_code = True
).to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [6]:
'''model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
).to(device) #'''

'model = AutoModelForCausalLM.from_pretrained(\n  model_name,\n  torch_dtype = torch.float16,\n  trust_remote_code = True\n).to(device) #'

## Import Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

## Model Preview

In [8]:
def assistant(prompt):
  prompt_format = f"### Human: {prompt}\n### Assistant :"

  inputs = tokenizer(
    prompt_format,
    return_tensors = 'pt'
  ).to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [9]:
# Level 1: Basic Prompt
prompt = "What are the symptoms and treatments of diabetes?"
assistant(prompt)

### Human: What are the symptoms and treatments of diabetes?
### Assistant : Diabetes is a chronic condition that affects the body's ability to produce or use insulin, a hormone that helps regulate blood sugar levels. There are two main types of diabetes: Type 1 and Type 2.

**Symptoms of Diabetes:**

1. **Type 1 Diabetes:**
	* Frequent urination
	* Increased thirst
	* Unexplained weight loss
	* Fatigue
	* Blurred vision
	* Slow-healing wounds
	* Numbness or tingling in hands or feet
2. **Type 2 Diabetes:**
	* Increased thirst
	* Frequent urination
	* Fatigue
	* Blurred vision
	* Slow-healing wounds
	* Numbness or tingling in hands or feet
	* Skin infections
	* Dark patches of skin (acanthosis nigricans)

**Treatments of Diabetes:**

1. **Type 1 Diabetes:**
	* Insulin therapy (injections or pumps)
	* Regular blood sugar monitoring
	* Healthy diet and exercise
	* Stress management
2. **Type 2 Diabetes:**
	* Lifestyle changes (diet, exercise, and weight loss)
	* Medications (oral medicat

In [10]:
# Level 2: Adding Detail
prompt = "Briefly describe the symptoms and treatments of diabetes."
assistant(prompt)

### Human: Briefly describe the symptoms and treatments of diabetes.
### Assistant : Diabetes is a chronic condition that affects the way the body processes blood sugar (glucose). The main symptoms of diabetes include frequent urination, increased thirst, extreme hunger, unexplained weight loss, fatigue, blurred vision, and slow-healing wounds. There are two main types of diabetes: Type 1 and Type 2. Type 1 diabetes is an autoimmune condition where the body's immune system attacks and destroys the insulin-producing cells in the pancreas. Type 2 diabetes is a condition where the body becomes resistant to insulin or doesn't produce enough insulin. Treatment for diabetes typically involves a combination of lifestyle changes, such as a healthy diet and regular exercise, and medication, such as insulin or oral medications to help regulate blood sugar levels. It's important for people with diabetes to work closely with their healthcare provider to manage their condition and prevent complicat

In [11]:
# Level 3: Specify Information Type and Audience
prompt = "Explain the common symptoms and treatments of diabetes to a general audience."
assistant(prompt)

### Human: Explain the common symptoms and treatments of diabetes to a general audience.
### Assistant : Diabetes is a chronic condition that affects the way your body processes blood sugar (glucose). There are two main types of diabetes: Type 1 and Type 2. Here's a simplified explanation of each type, along with common symptoms and treatments:

#### Type 1 Diabetes:
- **Symptoms:** Frequent urination, extreme thirst, constant hunger, unexplained weight loss, fatigue, blurred vision, and slow-healing wounds.
- **Causes:** The immune system mistakenly attacks and destroys the insulin-producing cells in the pancreas.
- **Treatments:** Insulin injections or an insulin pump to manage blood sugar levels, regular blood sugar monitoring, and a healthy diet and exercise plan.

#### Type 2 Diabetes:
- **Symptoms:** Similar to Type 1 but often less severe at first, including frequent urination, increased thirst, fatigue, blurred vision, and slow-healing wounds.
- **Causes:** Insulin resistance (

In [12]:
# Level 4: Request for Structured Response
prompt = "List the main symptoms of diabetes, then outline the standard treatments."
assistant(prompt)

### Human: List the main symptoms of diabetes, then outline the standard treatments.
### Assistant : The main symptoms of diabetes include frequent urination, increased thirst, extreme hunger, unexplained weight loss, fatigue, blurred vision, slow-healing sores, and frequent infections. The standard treatments for diabetes include lifestyle changes such as a healthy diet, regular exercise, and weight management. Medications such as insulin and oral medications may also be prescribed to help manage blood sugar levels. In some cases, surgery may be recommended for people with type 2 diabetes who have not responded to other treatments.


In [13]:
# Level 5: Differentiate by Type
prompt = "List the symptoms and treatments for both Type 1 and Type 2 diabetes."
assistant(prompt)

### Human: List the symptoms and treatments for both Type 1 and Type 2 diabetes.
### Assistant : Type 1 diabetes is an autoimmune disease in which the body's immune system attacks and destroys the insulin-producing beta cells in the pancreas. Symptoms of type 1 diabetes include frequent urination, excessive thirst, extreme hunger, unexplained weight loss, fatigue, blurred vision, and slow-healing wounds. Treatment for type 1 diabetes involves insulin therapy, which can be administered through injections or an insulin pump. Type 2 diabetes is a chronic condition in which the body becomes resistant to insulin or is unable to produce enough insulin to maintain normal blood sugar levels. Symptoms of type 2 diabetes include frequent urination, excessive thirst, extreme hunger, unexplained weight loss, fatigue, blurred vision, and slow-healing wounds. Treatment for type 2 diabetes includes lifestyle changes such as diet and exercise, oral medications, and in some cases, insulin therapy.


In [14]:
# Level 6: Target a Professional Audience
prompt = "Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know."
assistant(prompt)

### Human: Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know.
### Assistant : Type 1 diabetes is an autoimmune condition where the body's immune system attacks and destroys the insulin-producing beta cells in the pancreas. This results in a lack of insulin production, leading to high blood sugar levels. Symptoms include frequent urination, excessive thirst, unexplained weight loss, fatigue, and blurred vision. Diagnosis involves testing for autoantibodies and measuring blood glucose levels. Treatment includes insulin therapy, dietary management, and regular monitoring of blood sugar levels.

Type 2 diabetes is a chronic condition characterized by insulin resistance and relative insulin deficiency. It often develops gradually and may not have noticeable symptoms initially. Symptoms include increased thirst, frequent urination, fatigue, blurred vision, and slow-healing wounds. Diagnosis invo

In [15]:
# Level 7: Clinical Detail with Disease Stages
prompt = "Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages."
assistant(prompt)

### Human: Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages.
### Assistant : Diabetes is a chronic condition characterized by high blood sugar levels due to issues with insulin production or function. There are three main types of diabetes: prediabetes, Type 1, and Type 2.

**Prediabetes:**
- **Symptoms:** Often asymptomatic, but may include increased thirst, frequent urination, and fatigue.
- **Diagnostic Markers:** Fasting blood glucose levels between 100-125 mg/dL or HbA1c between 5.7-6.4%.
- **Treatments:** Lifestyle changes (diet, exercise), sometimes medication to improve insulin sensitivity.

**Type 1 Diabetes:**
- **Symptoms:** Rapid onset of symptoms such as extreme thirst, frequent urination, unexplained weight loss, and fatigue.
- **Diagnostic Markers:** Autoantibodies against insulin, low or undetectable insulin levels, and high blood 

In [16]:
# Level 8: Add Nuance with Treatment Personalization
prompt = "Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors."
assistant(prompt)

### Human: Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors.
### Assistant : Diabetes is a chronic condition characterized by high blood sugar levels due to either a lack of insulin production (Type 1) or the body's inability to use insulin effectively (Type 2). Here's a detailed overview of each type, including symptoms, diagnostic markers, and treatment options:

#### Type 1 Diabetes:
**Symptoms:**
- Frequent urination
- Extreme thirst
- Unexplained weight loss
- Fatigue
- Blurred vision
- Increased hunger

**Diagnostic Markers:**
- High blood glucose levels
- Presence of autoantibodies against insulin-producing cells (islet cells)
- Genetic testing for certain HLA types associated with Type 1 diabetes

**Treatment Options:**
- Insulin therapy (usually multiple daily injections or continuous subcutaneous insulin infusion)

In [17]:
# Level 9: Consider Risks, Complications, and Treatment Impacts
prompt = "Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment."
assistant(prompt)

### Human: Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment.
### Assistant : Diabetes is a chronic condition characterized by high blood sugar levels due to the body's inability to produce or properly use insulin. Insulin is a hormone that helps regulate blood sugar levels by allowing glucose to enter cells for energy production. There are two main types of diabetes: Type 1 and Type 2.

**Type 1 Diabetes:**
- **Symptoms:** Common symptoms include frequent urination, excessive thirst, extreme hunger, unexplained weight loss, fatigue, blurred vision, and slow-healing wounds.
- **Diagnostic Criteria:** Diagnosis is based on fasting blood glucose levels (126 mg/dL or higher), random blood glucose levels

In [18]:
# Level 10: Advanced Detail for Clinical Research or Specialist Training
prompt = "Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes."
assistant(prompt)

### Human: Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes.
### Assistant : Diabetes is a chronic metabolic disorder characterized by high blood glucose levels due to defects in insulin production, insulin action, or both. There are two main types of diabetes: Type 1 and Type 2.

**Type 1 Diabetes:**
- **Symptoms:** Polyuria, polydipsia, unexplained weight loss, fatigue, blurred vision, and slow healing of wounds.
- **Diagnostic Criteria:** Presence of autoantibodies (such as GAD65, ICA512, and IA-2), absence of insulin production, and a positive family history of Ty